In [1]:
ROOT_DIR = '/content'

In [2]:
!apt-get update
!apt-get install -y locales


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [908 kB]
Ge

In [3]:
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8


Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.


In [4]:
import os
os.environ['LC_ALL'] = 'C.UTF-8'
os.environ['LANG'] = 'C.UTF-8'


In [6]:
!pip install ultralytics PyYAML patool

In [7]:
# @title Check Dangerous Animals { display-mode: "form" }
zebra = False # @param {type:"boolean"}
lion = True # @param {type:"boolean"}
rhino = True # @param {type:"boolean"}
elephant = False # @param {type:"boolean"}
buffalo = False  # @param {type:"boolean"}
turtle = True  # @param {type:"boolean"}
ostrich = False  # @param {type:"boolean"}

# Download Dataset (Images and Labels)

In [8]:
%%bash

file_url="https://drive.google.com/uc?export=download&id=1FrPo0bICEH8Xwuyfl2TgBRycyQ6UYUtb"
destination_path="/content/wildlife_detector.zip"

confirmation_page=$(curl -s -L "$file_url")

file_id=$(echo "$confirmation_page" | grep -oE "name=\"id\" value=\"[^\"]+" | sed 's/name="id" value="//')
file_confirm=$(echo "$confirmation_page" | grep -oE "name=\"confirm\" value=\"[^\"]+" | sed 's/name="confirm" value="//')
file_uuid=$(echo "$confirmation_page" | grep -oE "name=\"uuid\" value=\"[^\"]+" | sed 's/name="uuid" value="//')

download_url="https://drive.usercontent.google.com/download?id=$file_id&export=download&confirm=$file_confirm&uuid=$file_uuid"

curl -L -o "$destination_path" "$download_url"

if [ $? -eq 0 ]; then
    echo "Download completed successfully."
else
    echo "Download failed."
fi

Download completed successfully.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  148M  100  148M    0     0  20.3M      0  0:00:07  0:00:07 --:--:-- 31.9M


In [11]:
!ls /content


__MACOSX  runs	sample_data  wildlife_detector	wildlife_detector.zip  yolov8n.pt


In [13]:
import zipfile

ROOT_DIR = "/content"  # Specify your desired output directory

zip_path = "/content/wildlife_detector.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(ROOT_DIR)


# Train YOLO Model with Custom Animal Dataset

In [16]:
from ultralytics import YOLO


def train_model():
  # Load a model
  model = YOLO("yolov8n.pt")  # build a pre-trained model


  # Use the model
  results = model.train(data=os.path.join(ROOT_DIR, "wildlife_detector", "config.yaml"), epochs=5)  # train the model

# Predict Animal Labels from Animal Videos

In [17]:
import os

from ultralytics import YOLO
import cv2

def predict_model(video_file='zebra.mp4'):
  VIDEOS_DIR = os.path.join(ROOT_DIR, "wildlife_detector", 'videos')

  video_path = os.path.join(VIDEOS_DIR, video_file)
  video_path_out = '{}_out.mp4'.format(video_path.split('.')[0])

  cap = cv2.VideoCapture(video_path)
  ret, frame = cap.read()
  H, W, _ = frame.shape
  out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

  model_path = os.path.join(ROOT_DIR, 'runs', 'detect', 'train', 'weights', 'best.pt')

  # Load a model
  model = YOLO(model_path)  # load a custom model

  threshold = 0.5

  while ret:

      results = model(frame)[0]

      for result in results.boxes.data.tolist():
          x1, y1, x2, y2, score, class_id = result

          if score > threshold:
              color = (0, 0, 255)
              if "NOT DANGEROUS" in results.names[int(class_id)].upper():
                color = (0, 255, 0)
              cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
              cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                          cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

      out.write(frame)
      ret, frame = cap.read()

  cap.release()
  out.release()
  cv2.destroyAllWindows()

# Pre-processed Sample Animal Videos

In [18]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/content/wildlife_detector/videos/turtles.mp4','rb').read()
zebra_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

mp4 = open('/content/wildlife_detector/videos/elephant_zebra_ostrich.mp4','rb').read()
lion_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

mp4 = open('/content/wildlife_detector/videos/rhino_zebra.mp4','rb').read()
rhino_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % (zebra_data_url, lion_data_url, rhino_data_url))

Output hidden; open in https://colab.research.google.com to view.

# Detect and Classify Animals in Videos using Model

In [19]:
import yaml
import shutil

def predict_animal_detector(buffalo, lion, zebra, rhino, elephant, turtle, ostrich):
  # Delete Previous Runs
  shutil.rmtree('/content/runs/', ignore_errors=True)

  # Update Config based on front-end input
  with open("/content/wildlife_detector/config.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    cfg['names']['0'] = 'buffalo (Dangerous)' if buffalo else 'buffalo (Not Dangerous)'
    cfg['names']['1'] = 'elephant (Dangerous)' if elephant else 'elephant (Not Dangerous)'
    cfg['names']['2'] = 'rhino (Dangerous)' if rhino else 'rhino (Not Dangerous)'
    cfg['names']['3'] = 'zebra (Dangerous)' if zebra else 'zebra (Not Dangerous)'
    cfg['names']['4'] = 'lion (Dangerous)' if lion else 'lion (Not Dangerous)'
    cfg['names']['5'] = 'ostrich (Dangerous)' if ostrich else 'ostrich (Not Dangerous)'
    cfg['names']['6'] = 'turtle (Dangerous)' if turtle else 'turtle (Not Dangerous)'

  with open("/content/wildlife_detector/config.yaml", "w") as f:
    cfg = yaml.dump(
        cfg, stream=f, default_flow_style=False, sort_keys=False
    )

  # Train YOLO Model with user input
  train_model()

  # Take 3 sample videos and use model to detect and classify animals
  for i in ['turtles.mp4', 'elephant_zebra_ostrich.mp4', 'rhino_zebra.mp4']:
    predict_model(i)

  # Convert video format to support browser
  os.system("ffmpeg -i /content/wildlife_detector/videos/turtles_out.mp4 -vcodec libx264 /content/wildlife_detector/videos/turtles_out_final.mp4")
  os.system("ffmpeg -i /content/wildlife_detector/videos/elephant_zebra_ostrich_out.mp4 -vcodec libx264 /content/wildlife_detector/videos/elephant_zebra_ostrich_out_final.mp4")
  os.system("ffmpeg -i /content/wildlife_detector/videos/rhino_zebra_out.mp4 -vcodec libx264 /content/wildlife_detector/videos/rhino_zebra_out_final.mp4")


predict_animal_detector(buffalo, lion, zebra, rhino, elephant, turtle, ostrich)

Ultralytics YOLOv8.2.73 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/wildlife_detector/config.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning /content/wildlife_detector/train/labels... 1565 images, 11 backgrounds, 0 corrupt: 100%|██████████| 1565/1565 [00:00<00:00, 2083.48it/s]

train: New cache created: /content/wildlife_detector/train/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/wildlife_detector/valid/labels... 283 images, 3 backgrounds, 0 corrupt: 100%|██████████| 283/283 [00:00<00:00, 872.34it/s]

...

        5/5      2.29G     0.8105      1.145      1.204         44        640: 100%|██████████| 98/98 [00:31<00:00,  3.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.54it/s]

                   all        283        440      0.922      0.701      0.869      0.672



5 epochs completed in 0.053 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.73 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:05<00:00,  1.74it/s]


                   all        283        440      0.921      0.703      0.869      0.672
buffalo (Not Dangerous)         62         89      0.959      0.796      0.949      0.737
elephant (Not Dangerous)         53         91      0.857      0.857      0.916      0.693
     rhino (Dangerous)         55         85      0.972      0.816      0.963      0.811
 zebra (Not Dangerous)         59        114      0.956      0.842      0.939      0.708
      lion (Dangerous)         46         51       0.85      0.941      0.879      0.684
ostrich (Not Dangerous)          3          3      0.853      0.667      0.913      0.747
    turtle (Dangerous)          6          7          1          0      0.522      0.322
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train

0: 384x640 3 elephant (Not Dangerous)s, 76.5ms
Speed: 1.1ms preprocess, 76.5ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 elephant

# Post-processing Sample Animal videos (Detection and Classification)

In [20]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/content/wildlife_detector/videos/turtles_out_final.mp4','rb').read()
zebra_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

mp4 = open('/content/wildlife_detector/videos/elephant_zebra_ostrich_out_final.mp4','rb').read()
lion_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

mp4 = open('/content/wildlife_detector/videos/rhino_zebra_out_final.mp4','rb').read()
rhino_data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % (zebra_data_url, lion_data_url, rhino_data_url))